<a href="https://colab.research.google.com/github/Gijslake/DDWT19/blob/master/skills_classificatie/model_tests/vacature_naar_meest_voorkomende_teksten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handmatige annotatie en scoring

## Milvus server inladen en starten

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Pip installaties

In [2]:
!pip install grpcio==1.58.0
!pip install pandas
!pip install milvus
!pip install pymilvus
!pip install pyyaml
!pip install scikit-learn
!pip install python-swiftclient python-keystoneclient
!pip install beautifulsoup4
!pip install lxml
!pip install sentence_transformers
!pip install jsonpickle
!pip install matplotlib
!pip install seaborn

get_ipython().kernel.do_shutdown(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.2
    Uninstalling grpcio-1.59.2:
      Successfully uninstalled grpcio-1.59.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{'status': 'ok', 'restart': True}

### Database laden vanuit drive

In [1]:
# Unzip milvus data van drive
!unzip /content/drive/MyDrive/Milvus_model/Milvus.zip -d /content

Archive:  /content/drive/MyDrive/Milvus_model/Milvus.zip
   creating: /content/Milvus/
   creating: /content/Milvus/logs/
  inflating: /content/Milvus/logs/milvus-stdout.log  
  inflating: /content/Milvus/logs/etcd.log  
  inflating: /content/Milvus/logs/milvus-stderr.log  
  inflating: /content/Milvus/logs/standalone-1.log  
  inflating: /content/Milvus/logs/standalone-1-2023-06-29T13-55-41.870.log  
  inflating: /content/Milvus/myeasylog.log  
   creating: /content/Milvus/configs/
  inflating: /content/Milvus/configs/milvus.yaml  
   creating: /content/Milvus/data/
   creating: /content/Milvus/data/etcd.data/
   creating: /content/Milvus/data/etcd.data/member/
   creating: /content/Milvus/data/etcd.data/member/snap/
  inflating: /content/Milvus/data/etcd.data/member/snap/db  
   creating: /content/Milvus/data/etcd.data/member/wal/
  inflating: /content/Milvus/data/etcd.data/member/wal/0000000000000001-000000000000d72f.wal  
  inflating: /content/Milvus/data/etcd.data/member/wal/00000

### Milvus server starten

In [2]:
# Milvus data inladen en starten
from milvus import default_server
default_server.set_base_dir('/content/Milvus')
default_server.config.set('proxy_port', 19531)
!chmod -R 777 /content/Milvus
default_server.start()

### Connectie maken met Milvus

In [3]:
from milvus import default_server
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
from time import sleep

# Wacht zodat de server opgestart is
sleep(30)

# Maak connection met server
connections.connect(
  "default",
  host=default_server.server_address,
  port=default_server.listen_port,
  timeout=30
)

In [4]:
# Check of connection gemaakt is
connections.list_connections()

[('default', <pymilvus.client.grpc_handler.GrpcHandler at 0x7ad6021ef250>)]

In [5]:
# Check of connection contact kan maken met de server
utility.list_collections(using='default')

['esco_db', 'onet_db', 'jobdigger_db', 'kwalificatie_db']

### Stop de server indien nodig

In [ ]:
# Stop your milvus server
#default_server.stop()

## Preparations

### Functies voor data ophalen en verwerken

In [6]:
import yaml

def read_secrets():
    with open("drive/MyDrive/InputData/secrets.yml", 'r') as file:
        data = yaml.safe_load(file)
        return data

secrets = read_secrets()
api_key = secrets['open-api']['key']
model = 'text-davinci-003'

In [7]:
def is_inline(item):
  # Returnt true als item tag een inline tag is
  inline_items = ["a", "abbr", "acronym", "b", "bdo", "big", "button", "cite",
                  "code", "dfn", "em", "i", "img", "input", "kbd", "label", "map",
                  "object", "output", "q", "samp", "script", "select", "small",
                  "span", "strong", "sub", "sup", "textarea", "time", "tt", "var"]
  return item.name in inline_items

In [8]:
import regex as re

def extract_text_recursive(item) -> [str]:
  pattern = r"^[^a-zA-Z0-9]+\s*"
  blocks = [""]

  # Itereer over alle block in een item
  for child in item.contents:

    # Als de name None is, gaat het om een bs4.element.NavigableString, dus een stuk tekst
    if child.name == None:
      blocks[-1] = blocks[-1] + " " + child.string.strip()

    # Als de child.name in de inline tags lijst staat, dan...
    elif is_inline(child):
      result = extract_text_recursive(child)
      # ... wordt de text behandeld als deel van het huidig block tenzij...
      if len(result) == 1:
        blocks[-1] = blocks[-1] + " " + result[0]
      # ... deze toch children blijkt te hebben, dan worden de children als aparte blokken behandeld
      else:
        blocks = blocks + result

    # Als het een block tag betreft, behandel het dan als nieuw block
    else:
      result = extract_text_recursive(child)
      # Als dit block daadwerkelijk tekst bevat, voeg het toe
      if len(result) > 0:
        blocks = blocks + result
      # Anders voeg een lege string toe om eventuele tekst uit children aan toe te voegen
      else:
        blocks.append("")

  # Verwijder lijsten met "- " en dergelijke en lege strings
  blocks = [re.sub(pattern, "", block.strip(), flags=re.S) for block in blocks if block.strip() != ""]
  return blocks

In [9]:
from sentence_transformers import SentenceTransformer

def load_model_from_drive(model_name = "model_file_all_skills_199k_15e_2023-06-27_11:39_best", load_name = "model_file_all_skills_199k_2023-06-27_11:39"):
  # Laad model vanuit model_file dir
  output_dir = "/content/"
  model_dir = f"/content/drive/MyDrive/models/{model_name}/model.zip"
  !unzip -o {model_dir} -d {output_dir}
  model = SentenceTransformer(output_dir + load_name)
  return model

In [10]:
def get_embedding_from_model(model, text):
  return model.encode(text, normalize_embeddings=True)

### Dictionary voor opzoeken data in Milvus

In [11]:
field_name_dict = {
    "jobdigger_db": {
        "id": "cluster_number",
        "text": "cluster_title_nl",
        "id_query": 'cluster_number > 0'
    }
    , "esco_db": {
        "id": "uri",
        "text": "total_text",
        "id_query": 'uri not in ["thisisnotanuri"]'
    }
    , "kwalificatie_db": {
        "id": "IDs",
        "text": "Text",
        "id_query": 'IDs not in ["thisisnotanID"]'
    }
    , "onet_db": {
        "id": "item_id",
        "text": "description_nl",
        "id_query": 'item_id not in ["thisisnotanID"]',
    }
}

# Laad collections naar memory en haal alle IDs op
for source in list(field_name_dict.keys()):
  field_name_dict[source]["collection"] = Collection(source)
  field_name_dict[source]["collection"].load()

## Vacatures laden, html code verwerken, scoren en schrijven naar bestand

In [12]:
import pickle
import csv
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
from random import choices
from sklearn.cluster import KMeans as sk_Kmeans
import time

# Kmeans model voor algemene clustering inladen
kmeans_general_model_path = "/content/drive/MyDrive/Osso data/clustering data/kmeans_model_all_skills.pickle"
with open(kmeans_general_model_path, 'rb') as file:
    kmeans_general = pickle.load(file)

general_clusters = [0, 4, 6, 13, 14, 15]

model = load_model_from_drive()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Archive:  /content/drive/MyDrive/models/model_file_all_skills_199k_15e_2023-06-27_11:39_best/model.zip
   creating: /content/model_file_all_skills_199k_2023-06-27_11:39/
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/config.json  
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/tokenizer_config.json  
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/tokenizer.json  
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/pytorch_model.bin  
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/modules.json  
   creating: /content/model_file_all_skills_199k_2023-06-27_11:39/eval/
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/eval/similarity_evaluation_results.csv  
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/unigram.json  
   creating: /content/model_file_all_skills_199k_2023-06-27_11:39/1_Pooling/
  inflating: /content/model_file_all_skills_199k_2023-06-27_11:39/1_Pooli

In [17]:
import swiftclient
import json
import swiftclient

# Read the settings from the JSON file
with open('drive/MyDrive/secrets_for_colab/osso_swift.json') as f:
    settings = json.load(f)

# Create a connection to the Swift API
conn = swiftclient.Connection(
    authurl=settings['auth_url'],
    user=settings['user_name'],
    key=settings['password'],
    os_options=settings['_os_options'],
    auth_version=settings['_auth_version']
)

# Set the container name and file name
container_name = 'testcontainer_dev'

# Get the files in the container
_, file_list = conn.get_container(container_name, prefix="matching_production/jobdigger")

In [22]:
import json

text_dict = {}
vac_max = 100000
vac_counter = 0

# Maak direrctory als die er niet is en zet de path variable
!mkdir -p data/tmp

for vacs_file in file_list:
  if vac_counter > vac_max:
        break

  # Haal file op
  name = vacs_file['name']
  _, obj_contents = conn.get_object(container_name, name)

  # Schrijf resultaat naar zip file
  with open("/content/data/tmp/tmp.json.zip", 'wb') as f:
    f.write(obj_contents)

  # Unzip
  file_in = "/content/data/tmp/tmp.json.zip"
  !unzip -o $file_in -d /content/data/tmp

  # Lees JSON file
  with open("/content/data/tmp/data.json", "rb") as f:
    data = json.load(f)
    for vac in data:
      if vac_counter > vac_max:
        break

      # Alleen groupleaders gebruiken
      if vac["PositionOpeningNL"]["PositionProfileNL"]["PositionDetail"]["Source"]["GroupLeader"] == "yes":
        if (vac_counter % 10000) == 0 and vac_counter > 0:
          print(f'{vac_counter} of {vac_max} ({round(vac_counter/vac_max * 100, 2)}%)')

        vac_counter += 1

        # HTML code opvragen
        html_code = list(vac["PositionOpeningNL"]["PositionProfileNL"]["PositionDetail"]["PositionTexts"].values())[0]

        # Parse HTML en split in kleinere stukjes
        soup = BeautifulSoup(html_code, "lxml")
        texts = extract_text_recursive(soup)

        for text in texts:
          if text in text_dict.keys():
            text_dict[text] += 1
          else:
            text_dict[text] = 1

Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
10000 of 100000 (10.0%)
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
Archive:  /content/data/tmp/tmp.json.zip
  inflating

In [29]:
from google.colab import data_table

results_df = pd.DataFrame(text_dict.items(), columns=['Text', 'Count'])
results_df = results_df.nlargest(10000, columns='Count')

In [30]:
display(data_table.DataTable(results_df, include_index=False, num_rows_per_page=10))

,Text,Count
1498,Dienstverband,18478
385,Functie-eisen,16001
461,Wat wij bieden,15640
12,Vacaturedetails,15280
4,Ontvang de nieuwste vacatures direct na plaats...,15274
...,...,...
555724,Als medewerker heb je recht op een tankkaart,19
567099,Making a difference,19
567889,Je zorgt er voor dat je afdeling er top uit ziet;,19
580291,2.750 tot € 3.350,19
